## The Movie Database API: Popularity by Movie Genre

* [Scraping the movie data](#Scraping_the_movie_data)
    * You can use the following API Key: **606aaffd7ca10f0b80804a1f0674e4e1** You can use the /discover/movie endpoint to get movies released over a period of time, as in the following example. Note use the total_pages parameter to paginate through the results on all pages. You can use the popularity and genre_ids attributes (among others) to answer these questions
* [Part 1.) Movie Popularity by genre](#Part1_Movie_Popularity_by_genre)
    * What was the most popular movie genre in December 2017? How did you define a metric for popularity, and why is it meaningful? What other metrics did you consider?
    * [Popular Movie Metric](#Popular_Metric)
    * [Movie Popularity By Genre Metric](#Popularity_By_Genre)
* [Part 2.) Analyze Popularity by Genre by time of year](#Part2_Popularity_by_Genre_by_time_of_year)
    * How does genre popularity vary by time of year? Why do you think this is?
    * [Data Visualization](#Visualization)
        * An interactive scatter plot (hover and zoom tool) showing genre popularity by time of year
    * [Summary of Results](#Summary_of_results)

<a id="Scraping_the_movie_data"></a>

### Scraping the movie data

In [516]:
import http.client

def get_data(action,primary_release_year,page_number,type1,type2):
    conn = http.client.HTTPSConnection("api.themoviedb.org")
    payload = "{}"
    API_KEY='606aaffd7ca10f0b80804a1f0674e4e1'
    conn.request("GET", "/3/{}?{}&{}&language=en-US&api_key={}".format(action,primary_release_year,page_number,API_KEY), payload)
    res = conn.getresponse()
    data = res.read()
    df=json_normalize(json.loads(data.decode('utf8'))['{}'.format(type1)])
    #df=df.drop(['backdrop_path','adult','poster_path','video'],1)
    total_pages=json.loads(data.decode('utf8'))['{}'.format(type2)]
    return df,total_pages


In [525]:
import time
from pandas.io.json import json_normalize

def get_multiple(action,primary_release_year):
    i = 0
    new_df=pd.DataFrame()
    '''
    #there's a 1000 maxmimum page limit
    if  get_data(action,1)[1]+1>1000:
        upper_limit=1001
    else:
        upper_limit=get_data(action,1)[1]+1 
    '''
    upper_limit=get_data(action,primary_release_year,1,'results','total_pages')[1]+1    
    for page in range(1,upper_limit):
        i +=1
        if (i%40 != 0):
            df=get_data(action,primary_release_year,'page={}'.format(page),'results','total_pages')[0]
            new_df=pd.concat([new_df,df],0).reset_index(drop=True)
        else:
            print('limit exceeded, sleep for 10 secs at page '+str(page))
            time.sleep(10)
    return new_df

#### Build movies Dataframe for release year 2017 using the function above

In [526]:
discover_movie=get_multiple('discover/movie','primary_release_year=2017')

limit exceeded, sleep for 10 secs at page 40
limit exceeded, sleep for 10 secs at page 80
limit exceeded, sleep for 10 secs at page 120
limit exceeded, sleep for 10 secs at page 160
limit exceeded, sleep for 10 secs at page 200
limit exceeded, sleep for 10 secs at page 240
limit exceeded, sleep for 10 secs at page 280
limit exceeded, sleep for 10 secs at page 320
limit exceeded, sleep for 10 secs at page 360
limit exceeded, sleep for 10 secs at page 400
limit exceeded, sleep for 10 secs at page 440
limit exceeded, sleep for 10 secs at page 480
limit exceeded, sleep for 10 secs at page 520
limit exceeded, sleep for 10 secs at page 560
limit exceeded, sleep for 10 secs at page 600
limit exceeded, sleep for 10 secs at page 640


C:\Users\scoll\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



limit exceeded, sleep for 10 secs at page 680
limit exceeded, sleep for 10 secs at page 720


<a id="Popular_Metric"></a>

Now let's construct our **Popularity** metric by taking the total votes count and adjusting it by the average vote. For example, let's say a movie was rated an 8.3 out of 10 and 1000 people voted for this movie, the **adj_voter_count** column takes 1000 * 83% = 830. The reason we wouldn't want to simply take 8.3 as a metric for popularity is because if a movie is ranked high but only a few people saw it, this doesn't reflect the magnitude. Conversely, if many people saw a bad movie but they didn't like it, we wouldn't want to define that movie as "popular" simply based on magnitude. So although "The Greatest Showman" received a higher voter average than Logan, not as many people went to go watch it(note: I'm using vote_count as a proxy for how many people watched the movie)

In [533]:
discover_movie['adj_voter_count']=discover_movie['vote_count']*(discover_movie['vote_average']/10)
discover_movie=discover_movie.sort_values('adj_voter_count',ascending=False).reset_index(drop=True)
discover_movie

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,adj_voter_count
0,False,/5pAGnkFYSsFJ99ZxDIYnhQbQFXs.jpg,"[28, 18, 878]",263115,en,Logan,"In the near future, a weary Logan cares for an...",34.149,/gGBu0hKw9BGddG8RkRAMX7B6NDB.jpg,2017-02-28,Logan,False,7.7,9853,7586.81
1,False,/6iUNJZymJBMXXriQyFZfLAKnjO6.jpg,"[28, 12, 14, 10752, 878]",297762,en,Wonder Woman,An Amazon princess comes to the world of Man i...,35.445,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,2017-05-30,Wonder Woman,False,7.2,9684,6972.48
2,False,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg,"[28, 12, 35, 878]",283995,en,Guardians of the Galaxy Vol. 2,The Guardians must fight to keep their newfoun...,34.306,/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,2017-04-19,Guardians of the Galaxy Vol. 2,False,7.6,9042,6871.92
3,False,/lAsr1ytUq0ATWdCbcjf59pupPFy.jpg,"[28, 12, 878, 18]",315635,en,Spider-Man: Homecoming,Following the events of Captain America: Civil...,34.488,/5x00yQZ3Mcz7duwMKc9GE8oZjFf.jpg,2017-07-05,Spider-Man: Homecoming,False,7.3,8343,6090.39
4,False,/6aUWe0GSl69wMTSWWexsorMIvwU.jpg,"[10751, 14, 10749]",321612,en,Beauty and the Beast,A live-action adaptation of Disney's version o...,27.472,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,2017-03-16,Beauty and the Beast,False,6.8,8870,6031.60
5,False,/tcheoA2nPATCm2vvXw2hVQoaEFD.jpg,"[18, 27, 53]",346364,en,It,"In a small town in Maine, seven children known...",18.654,/9E2y5Q7WlCVNEhP5GiVTjhEhx1o.jpg,2017-09-06,It,False,7.1,8400,5964.00
6,False,/kaIfm5ryEOwYg8mLbq8HkPuM1Fo.jpg,"[28, 12]",284053,en,Thor: Ragnarok,Thor is on the other side of the universe and ...,72.327,/rzRwTcFvttcN1ZpX2xv4j3tSdJu.jpg,2017-10-25,Thor: Ragnarok,False,7.5,7704,5778.00
7,False,/4G6FNNLSIVrwSRZyFs91hQ3lZtD.jpg,"[27, 53]",381288,en,Split,Though Kevin has evidenced 23 personalities to...,24.363,/rXMWOZiCt6eMX22jWuTOSdQ98bY.jpg,2017-01-19,Split,False,7.1,7190,5104.90
8,False,/2bG3HXcUze0GyGAKnJSDF6gllzk.jpg,"[28, 18, 36, 53, 10752]",374720,en,Dunkirk,The story of the miraculous evacuation of Alli...,26.463,/ebSnODDg9lbsMIaWg2uAbjn7TO5.jpg,2017-07-19,Dunkirk,False,7.4,6777,5014.98
9,False,/5OlAmzEUaO0A12cM7g5g420w4d7.jpg,"[9648, 53, 27]",419430,en,Get Out,Chris and his girlfriend Rose go upstate to vi...,26.863,/1SwAVYpuLj8KsHxllTF8Dt9dSSX.jpg,2017-02-24,Get Out,False,7.4,6151,4551.74


<a id="Popularity_By_Genre"></a>

### Get Popularity by movie genres

Now that we have a metric for popularity, we'll want to derive popularity by genre. First, I'll need to create genre columns. The way I'll about that is if the row contains a genre number then **adj_voter_count** else, 0. For example, for the column "Action" which corresponds to genre_id 28, notice that the first row of our dataframe above contains genre_id 28. So the first row of our "Action" would be 7586.81. However, the first row of our "Adventure" column would be 0 since the first row does not have a genre_id of 12

Let's begin by creating these genre features. First, we'll need to get a list of all of the genre_id's and their corresponding  names

In [534]:
genres=get_data('genre/movie/list',None,None,'genres','genres')[0]

In [535]:
genre_tuple=list(zip(list(genres['id']),list(genres['name'])))

In [537]:
genre_tuple

[(28, 'Action'),
 (12, 'Adventure'),
 (16, 'Animation'),
 (35, 'Comedy'),
 (80, 'Crime'),
 (99, 'Documentary'),
 (18, 'Drama'),
 (10751, 'Family'),
 (14, 'Fantasy'),
 (36, 'History'),
 (27, 'Horror'),
 (10402, 'Music'),
 (9648, 'Mystery'),
 (10749, 'Romance'),
 (878, 'Science Fiction'),
 (10770, 'TV Movie'),
 (53, 'Thriller'),
 (10752, 'War'),
 (37, 'Western')]

Now that we have all of the genre ids and names we can create the genre columns using the logic explained above

In [538]:
for i in range(0,len(genre_tuple)):
    discover_movie['{}'.format(genre_tuple[i][1])]=np.where(discover_movie['genre_ids'].map(lambda x: genre_tuple[i][0] in x),discover_movie['adj_voter_count'],0)

In [804]:
discover_movie[['genre_ids','original_title','popularity','vote_average','vote_count','adj_voter_count']+genre_names].sort_values('adj_voter_count',ascending=False)

,genre_ids,original_title,popularity,vote_average,vote_count,adj_voter_count,Action,Adventure,Animation,Comedy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,"[28, 18, 878]",Logan,34.149,7.7,9853,7586.81,7586.81,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,7586.81,0.0,0.00,0.00,0.0
1,"[28, 12, 14, 10752, 878]",Wonder Woman,35.445,7.2,9684,6972.48,6972.48,6972.48,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,6972.48,0.0,0.00,6972.48,0.0
2,"[28, 12, 35, 878]",Guardians of the Galaxy Vol. 2,34.306,7.6,9042,6871.92,6871.92,6871.92,0.00,6871.92,...,0.00,0.00,0.0,0.00,0.00,6871.92,0.0,0.00,0.00,0.0
3,"[28, 12, 878, 18]",Spider-Man: Homecoming,34.488,7.3,8343,6090.39,6090.39,6090.39,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,6090.39,0.0,0.00,0.00,0.0
4,"[10751, 14, 10749]",Beauty and the Beast,27.472,6.8,8870,6031.60,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,6031.60,0.00,0.0,0.00,0.00,0.0
5,"[18, 27, 53]",It,18.654,7.1,8400,5964.00,0.00,0.00,0.00,0.00,...,0.00,5964.00,0.0,0.00,0.00,0.00,0.0,5964.00,0.00,0.0
6,"[28, 12]",Thor: Ragnarok,72.327,7.5,7704,5778.00,5778.00,5778.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0
7,"[27, 53]",Split,24.363,7.1,7190,5104.90,0.00,0.00,0.00,0.00,...,0.00,5104.90,0.0,0.00,0.00,0.00,0.0,5104.90,0.00,0.0
8,"[28, 18, 36, 53, 10752]",Dunkirk,26.463,7.4,6777,5014.98,5014.98,0.00,0.00,0.00,...,5014.98,0.00,0.0,0.00,0.00,0.00,0.0,5014.98,5014.98,0.0
9,"[9648, 53, 27]",Get Out,26.863,7.4,6151,4551.74,0.00,0.00,0.00,0.00,...,0.00,4551.74,0.0,4551.74,0.00,0.00,0.0,4551.74,0.00,0.0


Now let's derive a column of yearmonth's

In [540]:
print('number of rows which do not have a release date associated with it= '+str(discover_movie[discover_movie['release_date']==''].shape[0])+' rows')

number of rows which do not have a release date associated with it= 0 rows


In [541]:
from datetime import datetime
discover_movie['movie_yrmo']=discover_movie['release_date'].map(lambda x: ((datetime.strptime(x, '%Y-%m-%d').year)*100)+datetime.strptime(x, '%Y-%m-%d').month)

<a id="Part1_Movie_Popularity_by_genre"></a>
### Part 1: Movie Popularity by genre(December 2017)

Now that we have genre popularity by movie, we could take the average score of each genre for December 2017. **However**, in order to consider the average, I will first need to account for the sparsity below

In [562]:
discover_movie[discover_movie['movie_yrmo']==201712][list(genres['name'])+['adj_voter_count']]

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,adj_voter_count
10,4424.72,4424.72,0.0,0.00,0.00,0.0,0.00,0.00,4424.72,0.00,0.00,0.00,0.00,0.00,4424.72,0.0,0.00,0.0,0.00,4424.72
15,0.00,0.00,0.0,0.00,0.00,0.0,3459.47,0.00,3459.47,0.00,0.00,0.00,0.00,3459.47,0.00,0.0,0.00,0.0,0.00,3459.47
18,3309.24,3309.24,0.0,3309.24,0.00,0.0,0.00,0.00,3309.24,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,3309.24
27,0.00,0.00,0.0,0.00,0.00,0.0,2372.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,2372.00
46,1392.93,0.00,0.0,0.00,1392.93,0.0,0.00,0.00,1392.93,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,1392.93
67,0.00,0.00,0.0,786.56,0.00,0.0,0.00,0.00,0.00,0.00,0.00,786.56,0.00,0.00,0.00,0.0,0.00,0.0,0.00,786.56
69,0.00,0.00,0.0,0.00,766.09,0.0,766.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,766.09
74,0.00,0.00,0.0,0.00,0.00,0.0,718.56,0.00,0.00,0.00,0.00,0.00,0.00,718.56,0.00,0.0,0.00,0.0,0.00,718.56
83,0.00,0.00,0.0,610.47,0.00,0.0,610.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00,610.47,0.0,0.00,0.0,0.00,610.47
84,0.00,608.30,608.3,608.30,0.00,0.0,0.00,608.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,608.30


In [572]:
yrmo=list(discover_movie['movie_yrmo'].unique())
genre_names=list(genres['name'])

The **pop_value** list below accounts for sparsity. Essentially, it calculates the average by genre and yrmo while ignoring rows containing zero. This gives a more accurate measure of the genres true average by yrmo

In [637]:
pop_value=[]
store_yrmo=[]
store_genre=[]
for i in yrmo:
    for j in genre_names:
        store_yrmo.append(str(i))
        store_genre.append(str(j))
        pop_value.append(discover_movie[discover_movie['movie_yrmo']==i][discover_movie[discover_movie['movie_yrmo']==i]['{}'.format(j)]!=0]['{}'.format(j)].sum()/discover_movie[discover_movie['movie_yrmo']==i][discover_movie[discover_movie['movie_yrmo']==i]['{}'.format(j)]!=0]['{}'.format(j)].count())

In [642]:
popularity_yrmo=pd.DataFrame({'yrmo': store_yrmo,'genre': store_genre,'popularity': pop_value})

In [647]:
popularity_yrmo[popularity_yrmo['yrmo']=='201712'].sort_values('popularity',ascending=False).reset_index(drop=True)

,yrmo,genre,popularity
0,201712,Fantasy,607.557143
1,201712,Adventure,597.547857
2,201712,Science Fiction,341.352000
3,201712,Action,261.134444
4,201712,Crime,114.230769
5,201712,Western,105.937500
6,201712,Drama,94.402523
7,201712,Romance,88.998800
8,201712,Comedy,66.932917
9,201712,Music,55.972667


In [648]:
discover_movie[discover_movie['movie_yrmo']==201712][discover_movie[discover_movie['movie_yrmo']==201712]['{}'.format('Fantasy')]!=0]['{}'.format('Fantasy')].sum()/discover_movie[discover_movie['movie_yrmo']==201712][discover_movie[discover_movie['movie_yrmo']==201712]['{}'.format('Fantasy')]!=0]['{}'.format('Fantasy')].count()

607.5571428571429

**Recap:** We considered an ajusted vote count metric as a measurement for popularity. We then essentially one-hot-encoded our genre categories where if row contained genre, then popularity metric, else 0. Lastly, we took an average of those scores(accounting for sparsity) and sorted them highest score to lowest

In [552]:
discover_movie.groupby(['movie_yrmo'],as_index=False)[list(genres['name'])].mean()

,movie_yrmo,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,201701,1.686696,1.517936,0.187781,1.732529,2.337409,0.142753,3.516956,0.543373,0.624755,0.018438,3.436832,0.016869,0.381847,0.774352,0.899361,0.037198,4.225456,0.012523,0.002480
1,201702,14.485969,0.249222,2.057503,4.843976,3.838258,0.490515,12.570712,1.981577,2.323056,0.174524,7.515181,0.361774,6.161457,3.111950,9.804524,0.054940,10.114688,0.166846,0.013713
2,201703,6.866235,4.394127,2.219839,4.771384,0.399928,0.404529,8.938842,6.988013,8.572494,1.639622,2.543315,0.204980,1.169558,5.741295,5.143379,0.136307,5.439920,0.092422,0.002011
3,201704,9.580740,9.136170,0.062272,8.784494,3.741191,0.324494,4.314757,1.135583,0.045481,0.288468,0.289362,0.115609,3.316340,0.640689,7.137549,0.038851,4.777923,0.355336,0.002451
4,201705,15.191191,11.414284,0.312412,7.249870,0.322282,0.300460,5.547357,0.144264,12.323514,0.278919,3.294585,0.188539,0.462563,1.564655,9.702322,0.109700,4.314925,7.576827,0.003203
5,201706,9.464388,5.269189,3.630229,5.775489,4.023163,0.441639,6.351242,3.684590,2.858899,0.025286,2.508626,0.463665,1.280291,2.131436,3.053771,0.021903,5.388581,0.232062,0.450731
6,201707,18.765317,10.638065,1.153978,3.109510,1.331481,0.173847,19.408029,1.505914,1.196738,6.489881,0.857658,0.477407,2.196941,0.887431,13.329797,0.499427,10.226894,9.531099,0.019319
7,201708,5.313013,0.445401,1.050089,5.146293,3.367354,0.107998,8.339514,0.268256,2.150218,0.095540,4.314460,0.024400,4.275540,0.336283,3.304034,0.062577,6.063380,0.002973,1.255619
8,201709,3.692075,2.342843,0.239373,2.713478,0.331940,0.445716,13.134545,0.269410,0.063791,0.981754,6.796269,0.274358,1.733716,2.729403,0.590761,0.059993,6.943015,0.012784,0.018022
9,201710,4.832368,4.859700,3.235342,1.467880,1.680946,0.249870,3.849987,0.421650,0.398467,0.302557,3.882381,0.400228,5.026151,0.670887,3.265264,0.052120,6.752922,0.081155,0.007626


<a id="Part2_Popularity_by_Genre_by_time_of_year"></a>

### Part 2: Popularity by Genre by time of year(2017)

In [754]:
#popularity_yrmo=popularity_yrmo[['yrmo','genre','popularity']].reset_index(drop=True)

In [755]:
colors=['red','blue','green','brown','purple','aqua','yellow','teal','plum','beige',
        'darkgreen','fuchsia','gray','black','palegoldenrod','peru','mediumpurple',
       'mediumaquamarine','mediumvioletred']

In [757]:
colors_df=pd.DataFrame({'color': colors,'genre': genre_names})
months=pd.DataFrame({'yrmo': list(popularity_yrmo['yrmo'].unique()),'month_name': ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']})

In [758]:
popularity_yrmo=popularity_yrmo.merge(colors_df,on='genre',how='left').merge(months,on='yrmo',how='left')
popularity_yrmo['month_number']=popularity_yrmo['yrmo'].map(lambda x : x[-2:])
popularity_yrmo=popularity_yrmo.sort_values('yrmo').reset_index(drop=True)

In [788]:
def check_data(release_yrmo,genre):
    df=discover_movie[discover_movie['movie_yrmo']==release_yrmo][discover_movie[discover_movie['movie_yrmo']==release_yrmo]['{}'.format(genre)]>0][['original_title','popularity','vote_average','vote_count','adj_voter_count']].sort_values('adj_voter_count',ascending=False)
    return df

In [809]:
import pandas as pd

from bokeh.io import output_file, show

import bokeh.plotting as bpl
from bokeh.plotting import figure
from bokeh.models import BoxSelectTool


import bokeh.models as bmo
#bpl.output_notebook()
from bokeh.io import output_notebook


<a id="Visualization"></a>


In [810]:
output_file("movie_genre_by_time_period.html")
output_notebook()

source = bpl.ColumnDataSource.from_df(popularity_yrmo)
hover = bmo.HoverTool(
    tooltips=[
        ('popularity', '@popularity'),
        ("genre", '@genre'),
        ('month','@month_name')
    ]
)
p = bpl.figure(tools=[hover,'wheel_zoom','pan','box_zoom'],
           title='2017 Movies by Genre',toolbar_location="below")

p.scatter(
    'month_number', 
    'popularity', source=source, color='color')


bpl.show(p)

Loading BokehJS ...

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: color [renderer: GlyphRenderer(id='61fdc1d2-ecf9-42b8-98e1-0c4cd8cfad9b', ...)]


<a id="Summary_of_results"></a>

From the chart above we can see that Adventure, Action and Science Fiction tend to be consistently one of the most popular genres by month. This might be attributed to all the marvel/superhero movies that were released in 2017. Documentaries tend to be the least desirable movie genre. We also notice seasonality as far as when people watch movies the most/least. For instance, people watch movies the least January and September while they watch the most movies during May and July. War was the most popular genre for both May and July. The function takes a further look at the data for War movies during May and July

The volume of when people see movies makes sense: work schedules tend to be more flexible during the summers and students are on vacation. Whereas students tend to start classes during September and January, which may explain the decrease of movie watchers during those months. May's "Wonder Woman" revenue was nearly 8 times it's original budget, moreover it was a much anticipated movie. As far as July, it had two popular war movies, "Dunkirk" and "War for the Planet of the Apes". There were less War movies than Action during that time, so although "Spider-Man: Homecoming" had a higher adj_voter_count(and Dunkirk was also considered an action movie) given that there were more undesirable action movies, movie goers were more interested in War over Action movies 		

In [792]:
check_data(201705,'War')

,original_title,popularity,vote_average,vote_count,adj_voter_count
1,Wonder Woman,35.445,7.2,9684,6972.48
155,War Machine,7.932,5.6,418,234.08
183,The Wall,9.425,6.0,296,177.60
236,The Man with the Iron Heart,7.926,6.4,188,120.32
374,Churchill,8.399,6.0,97,58.20
1416,대립군,1.929,8.1,7,5.67
3818,Ang Araw sa Likod Mo,0.451,9.0,1,0.90


In [793]:
check_data(201707,'War')

,original_title,popularity,vote_average,vote_count,adj_voter_count
8,Dunkirk,26.463,7.4,6777,5014.98
22,War for the Planet of the Apes,31.071,6.9,4155,2866.95
404,군함도,8.104,7.2,74,53.28
640,战狼2,6.420,6.8,36,24.48
918,City of Ghosts,3.339,7.2,17,12.24
1424,忍びの国,3.532,7.0,8,5.60


In [796]:
discover_movie[discover_movie['movie_yrmo']==201707][discover_movie[discover_movie['movie_yrmo']==201707]['{}'.format('adj_voter_count')]>0][['original_title','popularity','vote_average','vote_count','adj_voter_count']].sort_values('adj_voter_count',ascending=False)


,original_title,popularity,vote_average,vote_count,adj_voter_count
3,Spider-Man: Homecoming,34.488,7.3,8343,6090.39
8,Dunkirk,26.463,7.4,6777,5014.98
22,War for the Planet of the Apes,31.071,6.9,4155,2866.95
34,Valerian and the City of a Thousand Planets,19.828,6.6,3044,2009.04
38,Atomic Blonde,16.047,6.2,2840,1760.80
86,The Emoji Movie,14.671,5.5,1049,576.95
91,Girls Trip,9.906,6.8,786,534.48
93,Shot Caller,14.400,6.8,733,498.44
104,A Ghost Story,8.477,6.9,595,410.55
106,Detroit,12.152,7.3,554,404.42


In [798]:
discover_movie[discover_movie['original_title']=='Spider-Man: Homecoming']['genre_ids']

3    [28, 12, 878, 18]
Name: genre_ids, dtype: object

In [799]:
genre_tuple

[(28, 'Action'),
 (12, 'Adventure'),
 (16, 'Animation'),
 (35, 'Comedy'),
 (80, 'Crime'),
 (99, 'Documentary'),
 (18, 'Drama'),
 (10751, 'Family'),
 (14, 'Fantasy'),
 (36, 'History'),
 (27, 'Horror'),
 (10402, 'Music'),
 (9648, 'Mystery'),
 (10749, 'Romance'),
 (878, 'Science Fiction'),
 (10770, 'TV Movie'),
 (53, 'Thriller'),
 (10752, 'War'),
 (37, 'Western')]

In [800]:
check_data(201707,'Action')

,original_title,popularity,vote_average,vote_count,adj_voter_count
3,Spider-Man: Homecoming,34.488,7.3,8343,6090.39
8,Dunkirk,26.463,7.4,6777,5014.98
34,Valerian and the City of a Thousand Planets,19.828,6.6,3044,2009.04
38,Atomic Blonde,16.047,6.2,2840,1760.80
116,Descendants 2,9.061,7.3,497,362.81
292,First Kill,7.161,5.4,156,84.24
404,군함도,8.104,7.2,74,53.28
410,The Saint,9.588,5.2,100,52.00
423,Revolt,11.876,5.1,96,48.96
462,Stratton,7.760,4.7,90,42.30
